In [2]:
import torch.optim as optim
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
# read in data
from utils import get_matrix_features, generate_all_permutations, flatten_3d_array
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')


/home/farzaneh/miniconda3/envs/lineage_tracing/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# read data and save in matrix form
threshold_mode = 'count'
num_nn = 4
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'

colony0_features = pd.read_csv(os.path.join(
    path_to_data, 'colony0_candidate_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))
colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path).rename(
    columns={'parent_id': 'parent_GT'})
colony0_lineage_gt['colony'] = [0 for i in range(len(colony0_lineage_gt))]
colony0_matrix_features = get_matrix_features(
    colony0_features, colony0_lineage_gt)

# other colonies
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = pd.concat([colonies_gt, temp_colony])
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)

colonies_features = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    colonies_features = pd.concat([pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_candidate_features_{}_{}_nn.csv'.format(i, threshold_mode, num_nn))), colonies_features], ignore_index=True)

colonies_matrix_features = get_matrix_features(colonies_features, colonies_gt)


parent not in candidates 168 0 [ 33  77 159 154] 1
parent not in candidates 280 0 [105 212  27 238] 11
parent not in candidates 358 0 [-3 -3 -3 -3] 171
parent not in candidates 359 0 [-3 -3 -3 -3] 203
parent not in candidates 360 0 [-3 -3 -3 -3] 86
parent not in candidates 361 0 [-3 -3 -3 -3] 176
parent not in candidates 362 0 [-3 -3 -3 -3] 81
parent not in candidates 363 0 [-3 -3 -3 -3] 177
parent not in candidates 364 0 [-3 -3 -3 -3] 91
parent not in candidates 365 0 [-3 -3 -3 -3] 132
parent not in candidates 366 0 [-3 -3 -3 -3] 111
parent not in candidates 367 0 [-3 -3 -3 -3] 224
parent not in candidates 368 0 [-3 -3 -3 -3] 174
parent not in candidates 153 2 [-3 -3 -3 -3] 118
parent not in candidates 154 2 [-3 -3 -3 -3] 108
parent not in candidates 155 2 [-3 -3 -3 -3] 53
parent not in candidates 96 3 [-3 -3 -3 -3] 27
parent not in candidates 97 3 [-3 -3 -3 -3] 8
parent not in candidates 40 4 [-3 -3 -3 -3] 24
parent not in candidates 105 4 [-3 -3 -3 -3] 67
parent not in candidates 13

In [4]:
# read in data for colonies 1 to 5 and colony 0 and combine their matrices

# combine all data to make a single dataframe for all colonies
all_matrix_features = pd.concat(
    [colony0_matrix_features, colonies_matrix_features]).reset_index(drop=True)
all_matrix_features


,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
0,1,5,18,0,"[[2.23606797749979, 2.8284271247461903, 2.3462...","[1, 3, 2, 4]",0
1,3,6,18,0,"[[1.0, 3.0, 3.111078760931904, 3.0597171328670...","[3, 4, 2, 1]",0
2,2,7,20,0,"[[1.0, 3.0, 0.9423480319763352, 1.007736816321...","[2, 3, 4, 6]",0
3,4,8,22,0,"[[1.0, 2.0, 2.107542585550348, 2.1228587110116...","[4, 7, 6, 2]",0
4,1,9,32,0,"[[2.0, 2.0, 1.7589301904710015, 1.703711558127...","[1, 5, 3, 2]",0
...,...,...,...,...,...,...,...
821,22,34,164,5,"[[2.0, 3.0, 1.1800917718383046, 1.191595609891...","[22, 25, 8, 11]",0
822,13,35,166,5,"[[2.0, 3.0, 0.8067932399673147, 0.729620098766...","[13, 16, 10, 4]",0
823,18,36,169,5,"[[2.23606797749979, 4.123105625617661, 1.47672...","[18, 9, 2, 32]",0
824,3,37,175,5,"[[2.0, 2.23606797749979, 1.591168586220782, 1....","[18, 26, 3, 27]",2


In [5]:
from nn_model import BudDataset, LineageNN, train_nn, test_nn, cv_nn


# parameter tuning NN


In [14]:
layers_list = [[40,16,5], [40,5], [40,64,5], [40,64,16,5]]
lr_list = [0.01, 0.01, 0.001, 0.0001]
batch_size_list = [32, 128, 256]
patience = 10
epoch_n = 100
res_df = pd.DataFrame(columns=['layers', 'lr', 'batch_size', 'augment', 'patience', 'accuracy', 'std'])
for layers in layers_list:
    for lr in lr_list:
        for batch in batch_size_list:
            config = {'train_set': 'all', 'eval_set':'all', 'epoch_n': epoch_n, 'patience': patience, 'lr': lr, 'batch_size': batch, 'layers': layers, 'augment': True,}
            models, accuracies = cv_nn(all_matrix_features, config=config)
            row = pd.DataFrame({'layers': layers, 'lr': lr, 'batch_size': batch, 'augment': True, 'patience': patience, 'accuracy': np.mean(accuracies), 'std': np.std(accuracies)})
            res_df = pd.concat([row,res_df])
res_df.to_csv('nn_results.csv')
res_df.sort_values(by='accuracy', ascending=False).head(10)


config:  {'train_set': 'all', 'eval_set': 'all', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 32, 'layers': [40, 16, 5], 'augment': True, 'cv_number': 1}


best_accuracy,▁▄▄▄▄▅▅▅▅▇▇▇▇▇███████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
eval_accuracy,▁▄▄▄▄▅▄▅▅▇▆▇▆▇█▆▇▆▆▇▅█▇▆▅
patience,▁▁▂▁▂▁▂▁▂▁▂▂▃▁▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇▇▇███████████
best_accuracy,0.83636
epoch,24
eval_accuracy,0.78788
patience,10
train_accuracy,0.846


config:  {'train_set': 'all', 'eval_set': 'all', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 32, 'layers': [40, 16, 5], 'augment': True, 'cv_number': 2}


best_accuracy,▁▁▅▇▇▇▇▇▇▇███████████
epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
eval_accuracy,▅▂▇█▇▇▆▄▇▂█▃▅▇█▇▆▁█▁█
patience,▁▂▁▁▂▂▃▄▅▅▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▆▇▇▇▇▇█▇████████
best_accuracy,0.83735
epoch,20
eval_accuracy,0.83133
patience,10
train_accuracy,0.80581


config:  {'train_set': 'all', 'eval_set': 'all', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 32, 'layers': [40, 16, 5], 'augment': True, 'cv_number': 3}


...

best_accuracy,▁▂▄▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval_accuracy,▁▂▄▅▅▅▅▆▆▆▇▆▇▇▅▇▆▇▇▆▇▇▇█▇▇▇▇██▇▇█▇▇▇▇▇█▇
patience,▁▁▁▁▁▁▁▁▂▁▁▂▁▂▂▃▄▁▂▂▃▄▁▁▂▂▃▄▅▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████████████
best_accuracy,0.87349
epoch,39
eval_accuracy,0.84337
patience,10
train_accuracy,0.84306


config:  {'train_set': 'all', 'eval_set': 'all', 'epoch_n': 100, 'patience': 10, 'lr': 0.001, 'batch_size': 256, 'layers': [40, 64, 16, 5], 'augment': True, 'cv_number': 3}


best_accuracy,▁▂▃▃▃▃▄▄▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████████
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval_accuracy,▁▂▃▃▃▃▄▄▅▆▅▅▆▆▆▆▆▆▆▆▆▇▆▇▆▆▆▆▆▆█▇▇▆▇▇█▇█▇
patience,▁▁▁▂▂▃▁▂▁▁▂▃▄▁▂▂▃▄▅▅▇▁▂▁▂▂▃▄▅▅▁▂▂▃▄▅▅▆▇█
train_accuracy,▁▃▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇███████████████████
best_accuracy,0.85455
epoch,43
eval_accuracy,0.8303
patience,10
train_accuracy,0.84462


config:  {'train_set': 'all', 'eval_set': 'all', 'epoch_n': 100, 'patience': 10, 'lr': 0.001, 'batch_size': 256, 'layers': [40, 64, 16, 5], 'augment': True, 'cv_number': 4}


best_accuracy,▁▂▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇██████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval_accuracy,▁▂▄▅▅▅▅▅▆▅▆▆▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇▇█▇▇█▇▇█▇██▇
patience,▁▁▁▁▁▁▂▃▁▂▁▂▃▁▂▂▃▂▂▃▂▂▃▁▂▂▂▂▄▁▂▃▁▂▃▄▅▆▇█
train_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
best_accuracy,0.85455
epoch,53
eval_accuracy,0.83636
patience,10
train_accuracy,0.84304


In [ ]:
res_df.to_csv('nn_HPOptim.csv')

In [ ]:
# train on colonies and test of colony 0
train_df = colonies_matrix_features
test_df = colony0_matrix_features
config = {'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'save_path': 'bst_nn_4frames_10features_[40,64,5].pth'}
models, accuracies = cv_nn(train_df, config=config)
test_accuracies = []
for model in models:
    _ , accuracy = test_nn(model, test_df)
    test_accuracies.append(accuracy)
print('train on colony 1-5 and test on colony0\n ','accuracy: ', np.mean(test_accuracies), '+/-', np.std(test_accuracies))


# Train on all data and save the best model


In [8]:
# train and test on all data
config = {'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'save_path': 'bst_nn_01_256_[40,64,5].pth'}
models, accuracies = cv_nn(all_matrix_features, config=config)
test_accuracies = []
best_accuracy = 0.0
best_model = models[0]
for model in models:
    _ , accuracy = test_nn(model, all_matrix_features)
    test_accuracies.append(accuracy)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        print('new best accuracy: ', accuracy)
# save the best model
torch.save(best_model.state_dict(), 'bst_nn_all_4frames_10features_[40,64,5].pth')
print('average test accuracy on same data (all):' , np.mean(test_accuracies), '+/-', np.std(test_accuracies))
print("This model has been saved: ", best_accuracy)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


config:  {'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'save_path': 'bst_nn_01_256_[40,64,5].pth', 'cv_number': 1}


wandb: Currently logged in as: f-labbaf97. Use `wandb login --relogin` to force relogin


config:  {'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'save_path': 'bst_nn_01_256_[40,64,5].pth', 'cv_number': 2}


best_accuracy,▁▂▅▆▆▇▇▇▇▇▇███████████
epoch,▁▁▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇██
eval_accuracy,▁▂▅▆▆▇▇▇▇▇▆█▇▇▇█▇▇▇█▇▇
patience,▁▁▁▁▁▁▁▁▂▂▃▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▆▆▆▇▇▇▇▇▇██▇███████
best_accuracy,0.85542
epoch,21
eval_accuracy,0.83133
patience,10
train_accuracy,0.84628


...

config:  {'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'save_path': 'bst_nn_01_256_[40,64,5].pth', 'cv_number': 4}


best_accuracy,▁▂▃▃▃▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval_accuracy,▁▂▃▃▂▅▅▅▆▆▃▆▆▇▅▅▆▅▅▇▆▇▆▅▇▆▆▂▅█▇▆▇▇▇▆▇▇▇▇
patience,▁▁▁▁▂▁▂▁▁▂▂▃▄▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▄▅▅▆▇▇▇▇▇▇▇▇▇▇▇▇█▇▇███████▇████████████
best_accuracy,0.88485
epoch,39
eval_accuracy,0.85455
patience,10
train_accuracy,0.84909


config:  {'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'save_path': 'bst_nn_01_256_[40,64,5].pth', 'cv_number': 5}


best_accuracy,▁▃▃▃▄▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇███
eval_accuracy,▁▃▂▃▄▅▆▄▇▃▅▇▅▇▇▅▆▇▇▆▇█▇▇▅▆▆▄▇▆▆▆
patience,▁▁▂▂▁▁▁▂▁▂▂▁▂▂▃▄▅▅▆▇▇▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▇▇▇▇▇▇▇▇▇▇█████████████▇███
best_accuracy,0.85455
epoch,31
eval_accuracy,0.81212
patience,10
train_accuracy,0.84651


accuracy:  0.8619934282584885 +/- 0.01766325854316542
test accuracy 0.8353510895883777
new best accuracy:  0.8353510895883777
test accuracy 0.8631961259079903
new best accuracy:  0.8631961259079903
test accuracy 0.8426150121065376
test accuracy 0.847457627118644
test accuracy 0.8583535108958837
average test accuracy on same data (all): 0.8493946731234866 +/- 0.010175254910145625
This model has been saved:  0.8631961259079903


In [10]:
saved_path = '/home/farzaneh/Documents/Bread/bread/src/bread/algo/lineage/saved_models/bst_nn_all_4frames_10features_[40,64,5].pth'


# test saved model on external subset


In [12]:
external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'
external_subset_features_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/features/features.csv'
threshold_mode = 'count'
num_nn = 4
path_to_features = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/'

external_subset_features = pd.read_csv(os.path.join(
    path_to_features, 'external_subset_candidate_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))
external_subset_lineage_gt= pd.read_csv(external_subset_lineage_GT_path_edited)

# create matrix features for the external subset
external_subset_matrix_features = get_matrix_features(
    external_subset_features, external_subset_lineage_gt)

# load model from saved path
net = LineageNN(layers=[40, 64, 5])
net.load_state_dict(torch.load(saved_path))
# test on external subset
test_df = external_subset_matrix_features
test_df, accuracy = test_nn(net, test_df)


parent not in candidates 287 10 [269 207 122  92] -3
parent not in candidates 289 10 [269 207 197  61] -3
parent not in candidates 291 10 [269 207  92 122] -3
parent not in candidates 311 10 [-3 -3 -3 -3] 16
parent not in candidates 312 10 [-3 -3 -3 -3] 15
parent not in candidates 313 10 [-3 -3 -3 -3] 97
parent not in candidates 314 10 [-3 -3 -3 -3] 83
parent not in candidates 315 10 [-3 -3 -3 -3] 7
parent not in candidates 316 10 [-3 -3 -3 -3] 38
parent not in candidates 317 10 [-3 -3 -3 -3] 130
parent not in candidates 318 10 [-3 -3 -3 -3] 159
parent not in candidates 319 10 [-3 -3 -3 -3] 77
parent not in candidates 320 10 [-3 -3 -3 -3] 144
parent not in candidates 322 10 [-3 -3 -3 -3] 209
parent not in candidates 323 10 [-3 -3 -3 -3] 289
parent not in candidates 324 10 [-3 -3 -3 -3] 75
parent not in candidates 325 10 [-3 -3 -3 -3] 58
parent not in candidates 326 10 [-3 -3 -3 -3] 310
parent not in candidates 327 10 [-3 -3 -3 -3] 145
parent not in candidates 328 10 [-3 -3 -3 -3] 287
p

# Train on all data plus external data


In [7]:
# combine data from all colonies and external subset
external_subset_lineage_GT_path_edited = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/lineage_budlum_edited_by_code.csv'
external_subset_features_path = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/features/features.csv'
threshold_mode = 'count'
num_nn = 4
path_to_features = '/home/farzaneh/Documents/TrackerTestDataset_SCerevisiae_7/subset/lineage/'
external_subset_features = pd.read_csv(os.path.join(
    path_to_features, 'external_subset_candidate_features_{}_{}_nn.csv'.format(threshold_mode, num_nn)))
external_subset_lineage_gt= pd.read_csv(external_subset_lineage_GT_path_edited)

# create matrix features for the external subset
external_subset_matrix_features = get_matrix_features(
    external_subset_features, external_subset_lineage_gt)
external_subset_matrix_features

parent not in candidates 287 10 [269 207 122  92] -3
parent not in candidates 289 10 [269 207 197  61] -3
parent not in candidates 291 10 [269 207  92 122] -3
parent not in candidates 311 10 [-3 -3 -3 -3] 16
parent not in candidates 312 10 [-3 -3 -3 -3] 15
parent not in candidates 313 10 [-3 -3 -3 -3] 97
parent not in candidates 314 10 [-3 -3 -3 -3] 83
parent not in candidates 315 10 [-3 -3 -3 -3] 7
parent not in candidates 316 10 [-3 -3 -3 -3] 38
parent not in candidates 317 10 [-3 -3 -3 -3] 130
parent not in candidates 318 10 [-3 -3 -3 -3] 159
parent not in candidates 319 10 [-3 -3 -3 -3] 77
parent not in candidates 320 10 [-3 -3 -3 -3] 144
parent not in candidates 322 10 [-3 -3 -3 -3] 209
parent not in candidates 323 10 [-3 -3 -3 -3] 289
parent not in candidates 324 10 [-3 -3 -3 -3] 75
parent not in candidates 325 10 [-3 -3 -3 -3] 58
parent not in candidates 326 10 [-3 -3 -3 -3] 310
parent not in candidates 327 10 [-3 -3 -3 -3] 145
parent not in candidates 328 10 [-3 -3 -3 -3] 287
p

,Unnamed: 0,parent_GT,bud_id,time_index,colony,features,candidates,parent_index_in_candidates
3,3,1,4,6,10,"[[1.0, 4.0, 1.4643841598820833, 1.533490694990...","[1, 2, 3, -3]",0
4,4,2,5,8,10,"[[1.0, 2.8284271247461903, 1.8341048520519745,...","[2, 3, 1, 4]",0
5,5,3,6,9,10,"[[2.0, 2.23606797749979, 0.9843862591629124, 1...","[3, 5, 2, 1]",0
6,6,1,7,18,10,"[[1.0, 2.8284271247461903, 1.7695565443728838,...","[1, 4, 2, 3]",0
7,7,2,8,21,10,"[[1.0, 2.0, 1.8103577502210595, 1.819063025188...","[1, 2, 5, 3]",1
...,...,...,...,...,...,...,...,...
298,298,30,299,118,10,"[[1.0, 2.0, 2.091631799518852, 2.0103411040176...","[30, 52, 198, 239]",0
300,300,160,301,118,10,"[[1.0, 2.0, 0.5943675582008001, 0.662130820442...","[38, 160, 27, 90]",1
301,301,27,302,118,10,"[[1.0, 4.0, 4.516708573096808, 4.5093907953453...","[27, 90, 160, 38]",0
304,304,26,305,118,10,"[[1.0, 2.23606797749979, 0.9151683004515564, 1...","[26, 158, 237, 108]",0


In [8]:
all_plus = pd.concat([all_matrix_features, external_subset_matrix_features]).reset_index(drop=True)

In [10]:
# train on all plus external subset
config = {'train_set': 'all_plus', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True}
models, accuracies = cv_nn(all_plus, config=config)
test_accuracies = []
best_accuracy = 0.0
best_model = models[0]
for model in models:
    _ , accuracy = test_nn(model, all_plus)
    test_accuracies.append(accuracy)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model
        print('new best accuracy: ', accuracy)
# save the best model
torch.save(best_model.state_dict(), 'bst_nn_all_plus_4frames_10features_[40,64,5].pth')
print('average test accuracy on same data (all):' , np.mean(test_accuracies), '+/-', np.std(test_accuracies))
print("This model has been saved: ", best_accuracy)

config:  {'train_set': 'all_plus', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'cv_number': 1}


best_accuracy,▁▃▆▆▆███████████
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
eval_accuracy,▁▃▆▆▅█▇██▆█▇▇▇█▇
patience,▁▁▁▂▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▆▆▇▇▇█████████
best_accuracy,0.81333
epoch,15
eval_accuracy,0.8
patience,10
train_accuracy,0.84914


config:  {'train_set': 'all_plus', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'cv_number': 2}


best_accuracy,▁▁▃▅▆▆▆▇█████████████
epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
eval_accuracy,▁▁▃▅▆▄▅▇█▇██▇▇▇▇▇▇▇▇▇
patience,▁▂▁▁▁▂▂▁▁▂▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▇▇▇▇▇▇███████████
best_accuracy,0.82667
epoch,20
eval_accuracy,0.81333
patience,10
train_accuracy,0.85128


config:  {'train_set': 'all_plus', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'cv_number': 3}


...

config:  {'train_set': 'all_plus', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'cv_number': 4}


best_accuracy,▁▄▄▄▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
eval_accuracy,▁▄▄▃▆▅▆▅▆▆▇▇▇▆▆▆▇▆▆▇▆▅▅▆█▇█▅▅▅▅▃▆▇▄
patience,▁▁▂▂▁▂▁▂▂▁▁▂▂▃▄▅▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇█▇█████████████████
best_accuracy,0.82143
epoch,34
eval_accuracy,0.75
patience,10
train_accuracy,0.85386


config:  {'train_set': 'all_plus', 'epoch_n': 100, 'patience': 10, 'lr': 0.01, 'batch_size': 256, 'layers': [40, 64, 5], 'augment': True, 'cv_number': 5}


best_accuracy,▁▁▄▅▅▅▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇███████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
eval_accuracy,▂▁▄▅▆▄▇▄▅▂▃▇▆▅▇▇▇▆▇▇▇▆▇▇▇▇▆▄██▇▅▄▇▇▇▇▅▅
patience,▁▂▁▁▁▂▁▂▂▃▄▅▅▆▁▂▂▃▄▅▅▆▁▂▂▃▄▅▁▂▂▃▄▅▅▆▇▇█
train_accuracy,▁▅▅▆▆▇▇▇▇▇▇▇▇▇█▇████████████████▇██████
best_accuracy,0.83929
epoch,38
eval_accuracy,0.79464
patience,10
train_accuracy,0.85461


accuracy:  0.8465833333333335 +/- 0.022198713472149564
test accuracy 0.8465655664585192
new best accuracy:  0.8465655664585192
test accuracy 0.8296164139161463
test accuracy 0.848349687778769
new best accuracy:  0.848349687778769
test accuracy 0.8572702943800179
new best accuracy:  0.8572702943800179
test accuracy 0.8376449598572703
average test accuracy on same data (all): 0.8438893844781445 +/- 0.009474339373667649
This model has been saved:  0.8572702943800179
